<a href="https://colab.research.google.com/github/fedemoss/Clustering-of-Spotify-Songs/blob/main/spotify_API_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---
Este codigo se encarga de conectarme con la API de Spotify, pedirle mis Playlists y sacar las canciones de cada Playlist. En particular, esto se hace con la Playlist 'TOP 50 Argentina'. La razon de esto nace en que la idea de este proyecto era crear un algoritmo predictor de posibles canciones que puedan estar en el TOP 50. Este proyecto (Hobby) fue mutando y al final, descartamos la idea del predictor y nos enfocamos en Clusterizar grupos de canciones con features similares (Las features de una cancion las da Spotify: Bailabilidad, Nota, etc...). 

Ademas, este codigo se encarga de 'conseguir' letras de la famosa pagina de letras 'letras.com'. Para cada cancion de cada Playlist, sacamos la letra de dicha cancion y ponemos todo junto (cancion, features, letra) en un Dataframe. 

El analisis del Dataframe final lo analizaremos en otro Notebook. 

---
---

In [ ]:
# Importamos la librería requests para hacer peticiones a la API 
# y json para manipular ese tipo de datos
import requests 
import json
import numpy as np
from bs4 import BeautifulSoup as bs
import pandas as pd

# Connect to SPOTIFY

In [ ]:
# Claves obtenidas de Spotify.

CLIENT_ID = 'a0eb698c3613416685dbb50274e4437d'
CLIENT_SECRET = 'ac844b556d06485a9404fba52a9f839e'

# POST donde le pasamos las clave de la app
response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET})
print(response.status_code)

200


---

Bearer Token

---

In [ ]:
# Guardamos el bearer token para usarlo en las peticiones de la API
access_token = response.json()['access_token']
print('Access Token =',access_token)

Access Token = BQAAm4wuzgTaX-KzP7p6p3kCfduQXpttKBy2TXuMWXBPtLc6FCDLwMnRvdrvcqwV_ai7oAaaX_sYKr6CfZA



---

Esta clase nos permite llamar las funciones que nos interesan


Obs: tuve que modificarla. Me tiraba error siempre cuando la queria usar. El error estaba cuando queria correr la linea



```
len(json_data['items'])
```
Lo cambie por un np array y ahora todo anda bien. Es decir, hice 

```
np.size(json_data['items'])
```


Ademas, agregue varias features mas cuando pido get_songs_on_playlists ... 

In [ ]:
class spotydatos:
  def __init__(self, CLIENT_ID, CLIENT_SECTRET):
    self.client_id=CLIENT_ID
    self.client_secret=CLIENT_SECTRET
    self.access_token=requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': self.client_id, 'client_secret': self.client_secret}).json()['access_token']
    if response.status_code == 200:
      pass
    elif response.status_code != 200:
      print('ERROR, please verify ids (status code:{})'.format(response.status_code))

  def change_token(token):
    self.access_token=token

  def get_playslists(self,user,num=5):
    '''
    Returns information about the public playtlists of a given spotify user (No scope is required)
    Returns 4 lists: 
    1 Names of playlists
    2 Owner of playslist
    3 URL of playlist
    4 playlist id
    Arguement: "user" is the users name. "num" is the number of playlists to obtain. if the user has less than the asked number, all the playlists will be returned.
    '''
    playlists_names=[]
    owner_of_playlist=[]
    playlist_URL=[]
    playlist_id=[]
    headers = {'Authorization': 'Bearer {}'.format(self.access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
    url = 'https://api.spotify.com/v1/users/{}/playlists'.format(user)
    
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    for i in range(num):
      try:
        playlists_names.append(json_data['items'][i]['name'])
        owner_of_playlist.append(json_data['items'][i]['owner']['id'])
        playlist_URL.append(json_data['items'][i]['external_urls']['spotify'])
        playlist_id.append(json_data['items'][i]['id'])
      except:
        pass
    return  playlists_names,owner_of_playlist,playlist_URL,playlist_id


  def recently_played(self,auth_token,num=5):
    '''
    Return recently played songs information in form of lists, the most recent song is placed first
    The token must have the 'user-read-recently-played' scope authorization on the Spotify API
    Returns 4 lists:
    1 Song name
    2 Artist name
    3 Album name
    4 Song id
    Arguments: "auth_token" is the users access tocken, must have the 'user-read-recently-played' scope authorized. "num" is the maximum number of items to return, maximum: 50.
    '''
    song_name=[]
    artist_name=[]
    album_name=[]
    song_id=[]
    headers = {'Authorization': 'Bearer {}'.format(auth_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
    url = 'https://api.spotify.com/v1/me/player/recently-played?limit={}'.format(num)
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    if len(json.dumps(json_data, indent = 4))<200:
      print(json.dumps(json_data, indent = 4))
    elif len(json.dumps(json_data, indent = 4))>200:
      for i in range(num):
        song_name.append(json_data['items'][i]['track']['name'])
        artist_name.append(json_data['items'][i]['track']['artists'][0]['name'])
        album_name.append(json_data['items'][i]['track']['album']['name'])
        song_id.append(json_data['items'][i]['track']['id'])
    return song_name,artist_name,album_name,song_id

  def get_song_features(self,track_id):
    '''
    Return a dict of features of a given song id. No scope required
    dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'duration_ms', 'time_signature'])
    ''' 
    features={}
    headers = {'Authorization': 'Bearer {}'.format(self.access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
    url = 'https://api.spotify.com/v1/audio-features/{}'.format(track_id)
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    features_list=json.dumps(json_data, indent = 4).replace('\n','').replace(' ','').replace('{','').replace('}','').split(',')
    keys=[]
    values=[]
    for i in range(np.size(features_list)):
      keys.append(features_list[i].split(':')[0].replace('"',''))
      values.append(features_list[i].split(':')[1].replace('"',''))
    for j in range(np.size(keys)):
      try:
        values[j]=float(values[j])
        features['{}'.format(keys[j])]=values[j]
      except:
        features['{}'.format(keys[j])]='{}'.format(values[j])
        
    del features['analysis_url'] #dropea estas features 
    del features['track_href']
    del features['uri']
    return features

  def get_songs_on_playlist(self,playlist_id,only_id=True):
    '''
    Returns information about the songs on a given playlist.
    If the only_id is set to True (default), only the ids of the songs on that playlists are returned. If only_id is False, then the function returns a list
    of dictionaries, each with data about one song in the playlist  
    '''
    headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
    playlist_id=playlist_id
    url = 'https://api.spotify.com/v1/playlists/{}/tracks'.format(playlist_id)
    response = requests.get(url, params = {}, headers = headers)
    json_data = response.json()
    if only_id==True:
      ids=[]
      for i in range(np.size(json_data['items'])):
        ids.append(json_data['items'][i]['track']['id'])
      return ids
    elif only_id==False:
      info=[]
      for i in range(np.size(json_data['items'])):
        track_dict={}
        track_dict['Track number']=i+1
        track_dict['Track name'] = json_data['items'][i]['track']['name']
        track_dict['Album'] = json_data['items'][i]['track']['album']['name']
        track_dict['Artist'] = json_data['items'][i]['track']['album']['artists'][0]['name']
        track_dict['id'] = json_data['items'][i]['track']['id']
        track_dict['Popularity'] = json_data['items'][i]['track']['popularity'] #Agrego features
        track_dict['Disc number'] = json_data['items'][i]['track']['disc_number']
        track_dict['Track number'] = json_data['items'][i]['track']['track_number']
        track_dict['Total tracks'] = json_data['items'][i]['track']['album']['total_tracks']
        track_dict['Date'] = json_data['items'][i]['track']['album']['release_date']


        info.append(track_dict.copy())
      return info


Saquemos las canciones de alguna playlist que haya creado

In [ ]:
# Acá le pasamos el token que obtuvimos antes. Lo demás son algunas cosas generales que suelen ir 
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
user='11169876763'
url = 'https://api.spotify.com/v1/users/{}/playlists?limit=30&offset=0'.format(user)

# En este caso no lleva ningún parámetro, el id de la canción va directamente en el url
response = requests.get(url, params = {}, headers = headers)
json_data = response.json()
# Vemos el json de la respuesta
json_data.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [ ]:
#elijamos alguna de mis playlists
len_json = np.size(json_data['items'])

for i in range(len_json):
  print(i,json_data['items'][i]['name'])

0 Coconut Radio
1 Rock Argentino años 60
2 ROCK NACIONAL ARGENTINO || 2020
3  ROCK ARGENTINO 🔥 : Rock Nacional Argentino
4 Rock argentino 2000-2020
5 Rock argentino 2000
6 Rock Nacional Argentino 2000s
7 Rock Argentino 60’70’80’
8 ROCK ARGENTINO 70's
9 Rock Argentino 80's
10 Rock argentino 90's
11 Rock Nacional 80 y 90's
12 Mix rock nacional 80'90'
13 Rock nacional 60 y 70
14 Rock nacional '60 y '70 (y algo de los '80)
15 Rock nacional 60 70 80
16 Rock Nacional de los 70
17 Rock nacional 2010-2019
18 Rock Nacional 2000 - 2010
19 Rock Nacional 90's ARG
20 Rock Nacional 80's ARG
21 Rock Nacional 70/80’
22  ROCK NACIONAL ARGENTINO 
23 Best Classical Music
24 Guardians of the Galaxy Vol. 1&2 
25 Antidomingo
26 Top 50 - Argentina
27 Rock nac
28 Disparo al corazón
29 Shazam


Muchas no son mias, lo juro. Elijamos alguna. Vayamos al grano: elijamos el TOP 50 argentina.

OBS: Para que aparezcan aca, hay que darles 'me gusta' en la app y hacerlas publicas!

In [ ]:
#Elijamos la playlist del TOP 50 ARGENTINA
nro = 26 #nro de playlist

playlist_id = json_data['items'][nro]['id']

print('Playlist id:',json_data['items'][nro]['id'])

Playlist id: 37i9dQZEVXbMMy2roB9myp


In [ ]:
#las tracks ID, nombres y artistas de las canciones de la playlist
analyzer = spotydatos(CLIENT_ID,CLIENT_SECRET)

songs = analyzer.get_songs_on_playlist(playlist_id,only_id=False)
songs_names = []
songs_artists = []
songs_id = []
songs_total_tracks = []
songs_track_number = []
songs_disc_number = []
songs_popularity = []
songs_date = []

songs_names_raw = [] #esto me va a servir para mas tarde
songs_artists_raw = [] #esto me va a servir para mas tarde


for i in range(np.size(songs)):
  songs_names.append(songs[i]['Track name'])
  songs_names_raw.append(songs[i]['Track name'])

  songs_artists.append(songs[i]['Artist'])
  songs_artists_raw.append(songs[i]['Artist'])

  songs_id.append(songs[i]['id'])
  songs_total_tracks.append(songs[i]['Total tracks'])
  songs_track_number.append(songs[i]['Track number'])
  songs_disc_number.append(songs[i]['Disc number'])
  songs_popularity.append(songs[i]['Popularity'])
  songs_date.append(songs[i]['Date'])

print(songs_artists)
print(songs_names)

['Kaleb Di Masi', 'Maria Becerra', 'Tiago PZK', 'Mau y Ricky', 'Emilia', 'Farruko', 'Marama', 'Emilia', 'BANDIDO', 'Maria Becerra', 'Kaleb Di Masi', 'J Balvin', 'Bizarrap', 'Tiago PZK', 'Rauw Alejandro', 'Kaleb Di Masi', 'Rusherking', 'TINI', 'MYA', 'Aventura', 'Bizarrap', 'Tainy', 'Rauw Alejandro', 'Ingratax', 'Maluma', 'Mora', 'Trueno', 'Bad Bunny', 'Duki', 'Kaleb Di Masi', 'FMK', 'TINI', 'Bruno LC', 'Rvssian', 'KAROL G', 'MYA', 'Bizarrap', 'J Balvin', 'Perro Primo', 'Nio Garcia', 'FMK', 'Danny Ocean', 'LIT killah', 'Nicki Nicole', 'LIT killah', 'Marc Seguí', 'Sebastian Yatra', 'Rusherking', 'Myke Towers', 'LIT killah']
['Turraka - Remix', 'Wow Wow (feat. Becky G)', 'Entre Nosotros', 'Mal Acostumbrao', 'Rápido Lento', 'Pepas', 'Ya No Llora', 'Como Si No Importara', 'No Me Conocen (Remix) [con DUKI, Rei & Tiago PZK]', 'Mi Debilidad', 'Matatan', 'Qué Más Pues?', 'Ptazeta: Bzrp Music Sessions, Vol. 45', 'Loco', 'Cúrame', 'KALEB DI MASI | DJ TAO Turreo Sessions #5', 'Antes de Ti', 'Miént

In [ ]:
#las features de las canciones
features = []
for i in range(np.size(songs_id)): #de nuevo, len(list) no funca! No se que anda pasando... 
  features.append(analyzer.get_song_features(songs_id[i]))

  print(analyzer.get_song_features(songs_id[i]))
 

{'danceability': 0.799, 'energy': 0.6, 'key': 7.0, 'loudness': -7.474, 'mode': 1.0, 'speechiness': 0.332, 'acousticness': 0.39, 'instrumentalness': 0.0, 'liveness': 0.0474, 'valence': 0.683, 'tempo': 170.22, 'type': 'audio_features', 'id': '2Qv0DF1us4LhzLj0Cx9cSn', 'duration_ms': 210353.0, 'time_signature': 4.0}
{'danceability': 0.744, 'energy': 0.679, 'key': 5.0, 'loudness': -3.51, 'mode': 1.0, 'speechiness': 0.0791, 'acousticness': 0.0654, 'instrumentalness': 0.00429, 'liveness': 0.0966, 'valence': 0.866, 'tempo': 176.024, 'type': 'audio_features', 'id': '4Yip3hds5IHyT7AqoNHfmX', 'duration_ms': 199773.0, 'time_signature': 4.0}
{'danceability': 0.729, 'energy': 0.513, 'key': 5.0, 'loudness': -6.345, 'mode': 0.0, 'speechiness': 0.0455, 'acousticness': 0.494, 'instrumentalness': 1.1e-06, 'liveness': 0.137, 'valence': 0.789, 'tempo': 170.064, 'type': 'audio_features', 'id': '6w8yBI2vthyN9UnwO4UBWb', 'duration_ms': 192353.0, 'time_signature': 3.0}
{'danceability': 0.86, 'energy': 0.516, '

# Web Scrapping LETRAS


---
Ahora usemos lo de web Scrapping para sacar la letra de cada tema

In [ ]:
url_base = 'https://www.musica.com/letras.asp?t2='
urls = []
songs_names_clean = []

for song in range(np.size(songs_names)):
  songs_names[song] = songs_names[song].replace(' ','+').replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('Á','a').replace('É','e').replace('Í','i').replace('Ó','o').replace('Ú','u').lower()
  songs_artists[song] = songs_artists[song].replace(' ','+').replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('Á','a').replace('É','e').replace('Í','i').replace('Ó','o').replace('Ú','u').lower()
  split_1 = songs_names[song].split(':',1) #sacamos los (feat), remix, :, ...
  split_2 = split_1[0].split('+(',1)
  split_3 = split_2[0].split('(',1)
  split_4 = split_3[0].split('remix',1)
  split_5 = split_4[0].split('+-',1)
  songs_names_clean.append(split_5[0]) 
  #print(song, split_4[0])
  
  urls.append([url_base + songs_names_clean[song] +'+'+ songs_artists[song]])

print(urls)


[['https://www.musica.com/letras.asp?t2=turraka+kaleb+di+masi'], ['https://www.musica.com/letras.asp?t2=wow+wow+maria+becerra'], ['https://www.musica.com/letras.asp?t2=entre+nosotros+tiago+pzk'], ['https://www.musica.com/letras.asp?t2=mal+acostumbrao+mau+y+ricky'], ['https://www.musica.com/letras.asp?t2=rapido+lento+emilia'], ['https://www.musica.com/letras.asp?t2=pepas+farruko'], ['https://www.musica.com/letras.asp?t2=ya+no+llora+marama'], ['https://www.musica.com/letras.asp?t2=como+si+no+importara+emilia'], ['https://www.musica.com/letras.asp?t2=no+me+conocen+bandido'], ['https://www.musica.com/letras.asp?t2=mi+debilidad+maria+becerra'], ['https://www.musica.com/letras.asp?t2=matatan+kaleb+di+masi'], ['https://www.musica.com/letras.asp?t2=que+mas+pues?+j+balvin'], ['https://www.musica.com/letras.asp?t2=ptazeta+bizarrap'], ['https://www.musica.com/letras.asp?t2=loco+tiago+pzk'], ['https://www.musica.com/letras.asp?t2=curame+rauw+alejandro'], ['https://www.musica.com/letras.asp?t2=kale

In [ ]:
# Hacemos un request iterativo a la página de musica.com y sacamos como output los urls a las letras
url_lyrics = []

for url in np.arange(np.size(urls)):
  try:
    search = requests.get(urls[url][0])
    search_parseada = bs(search.content, 'html.parser') # Parseamos el contenido del request como un html
    # Soup
    soup = bs(response.content)
    tag_song = search_parseada.find(name = 'table', attrs = {'class' : 'rst'}) #tag madre
    tag_song = tag_song.find(name = 'a') # tag en donde esta el link a la letra
    url_lyrics.append(tag_song['href']) # el link en cuestion
    print(url)
  except:
    url_lyrics.append(np.NaN) #le asigno un NaN
    print(url, 'Lyrics not found')


0 Lyrics not found
1
2
3
4
5
6 Lyrics not found
7
8 Lyrics not found
9
10 Lyrics not found
11
12
13
14
15 Lyrics not found
16
17
18
19
20
21
22
23 Lyrics not found
24
25
26
27
28
29 Lyrics not found
30
31
32 Lyrics not found
33
34
35
36
37
38 Lyrics not found
39
40 Lyrics not found
41
42
43
44
45
46
47
48
49


Se ve a priori que no encuentra todas las url de las letras... esto es porque no le estamos dando el nombre del artista/cancion que le gusta a musica.com

Ahora iteramos sobre las url en donde estan las letras.. Nuevamente, proponemos un try/except porque puede pasar que el url encontrado en el loop anterior no de el link correcto. 

In [ ]:
#iteramos sobre toda las urls de las letras
lyrics = []
for url in np.arange(np.size(url_lyrics)):
  try:
    search_song = requests.get(url_lyrics[url]) #pedimos el request
    search_song_parseada = bs(search_song.content, 'html.parser')

    body = search_song_parseada.find(name = 'div', attrs = {'id' : 'letra'}) #aca esta la letra, entre otras cosas

    letra_raw = body.find_all('p') #buscamos los paragraphs
    letra_raw_string = str(letra_raw) #to string
    letra_raw_string = str(letra_raw).replace('<br/>',' ').replace('</p>','').replace('<p>','') #mas bonita
    lyrics.append(letra_raw_string)
    print(url)
  except:
    lyrics.append(np.NaN)
    print(url,'Lyrics not found')


0 Lyrics not found
1
2
3
4
5
6 Lyrics not found
7
8 Lyrics not found
9
10 Lyrics not found
11
12
13
14
15 Lyrics not found
16
17
18 Lyrics not found
19
20
21
22
23 Lyrics not found
24
25
26
27
28
29 Lyrics not found
30 Lyrics not found
31
32 Lyrics not found
33
34
35
36
37
38 Lyrics not found
39 Lyrics not found
40 Lyrics not found
41
42
43
44
45
46
47
48
49


---

Metamos las lyrics en un dataframe. Despues podemos hacer un merge con otro dataframe de las features de cada cancion.

In [ ]:
#Dataframe

df_lyrics_top = pd.DataFrame(list(zip(songs_names_raw,songs_artists_raw,lyrics)),index=songs_id)
df_lyrics_top['TOP'] = 1 #esta en el top

df_lyrics_top.head()

,0,1,2,TOP
2Qv0DF1us4LhzLj0Cx9cSn,Turraka - Remix,Kaleb Di Masi,NaN,1
4Yip3hds5IHyT7AqoNHfmX,Wow Wow (feat. Becky G),Maria Becerra,"[(This is the Big One), Una vez me desperté (M...",1
6w8yBI2vthyN9UnwO4UBWb,Entre Nosotros,Tiago PZK,[Hace un tiempo no duermo con vos Pienso cómo ...,1
1wcE84DVQUF16xudVFzezL,Mal Acostumbrao,Mau y Ricky,[Sí Dime que sientes Qué te imaginas Qué te pa...,1
4xv2A022v5A8nyewDkYOkQ,Rápido Lento,Emilia,"[Vos y yo (Mmm) Tenemos algo único, mmm (Yeah-...",1


In [ ]:
#YAPA: dataframe de features, canciones y artistas
df_features_top = pd.DataFrame(features, index=songs_id)
df_features_top.drop(df_features_top['id'])
df_features_top['Artist'] = songs_artists_raw
df_features_top['Song'] = songs_names_raw
df_features_top['Disc number'] = songs_disc_number
df_features_top['Total tracks'] = songs_total_tracks
df_features_top['Track number'] = songs_track_number
df_features_top['Date'] = songs_date
df_features_top['Popularity'] = songs_popularity
df_features_top['TOP'] = 1
df_features_top.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,duration_ms,time_signature,Artist,Song,Disc number,Total tracks,Track number,Date,Popularity,TOP
2Qv0DF1us4LhzLj0Cx9cSn,0.799,0.600,7.0,-7.474,1.0,0.3320,0.3900,0.000000,0.0474,0.683,170.220,audio_features,2Qv0DF1us4LhzLj0Cx9cSn,210353.0,4.0,Kaleb Di Masi,Turraka - Remix,1,1,1,2021-09-09,79,1
4Yip3hds5IHyT7AqoNHfmX,0.744,0.679,5.0,-3.510,1.0,0.0791,0.0654,0.004290,0.0966,0.866,176.024,audio_features,4Yip3hds5IHyT7AqoNHfmX,199773.0,4.0,Maria Becerra,Wow Wow (feat. Becky G),1,11,7,2021-08-26,72,1
6w8yBI2vthyN9UnwO4UBWb,0.729,0.513,5.0,-6.345,0.0,0.0455,0.4940,0.000001,0.1370,0.789,170.064,audio_features,6w8yBI2vthyN9UnwO4UBWb,192353.0,3.0,Tiago PZK,Entre Nosotros,1,1,1,2021-07-08,89,1
1wcE84DVQUF16xudVFzezL,0.860,0.516,6.0,-5.518,0.0,0.1450,0.0616,0.000000,0.0791,0.591,98.005,audio_features,1wcE84DVQUF16xudVFzezL,190406.0,4.0,Mau y Ricky,Mal Acostumbrao,1,1,1,2021-09-02,80,1
4xv2A022v5A8nyewDkYOkQ,0.793,0.352,8.0,-7.675,1.0,0.0955,0.3020,0.000000,0.1750,0.666,80.014,audio_features,4xv2A022v5A8nyewDkYOkQ,213000.0,4.0,Emilia,Rápido Lento,1,1,1,2021-09-30,70,1


---

Ahora, para aumentar la cantidad de datos, vamos a agarrar otras playlist con canciones que ya se que no estan en el top y vamos a crear otro dataframe.


OBS: Descargue mas cantidad de playlists para hacer estadistica. 

In [ ]:
#elijamos alguna otra de mis playlists

# Claves obtenidas de Spotify.

CLIENT_ID = 'a0eb698c3613416685dbb50274e4437d'
CLIENT_SECRET = 'ac844b556d06485a9404fba52a9f839e'
response = requests.post('https://accounts.spotify.com/api/token', data = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET})
access_token = response.json()['access_token']

# Acá le pasamos el token que obtuvimos antes. Lo demás son algunas cosas generales que suelen ir 
headers_ = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
user_='11169876763'
url_ = 'https://api.spotify.com/v1/users/{}/playlists?limit=50&offset=0'.format(user_)

# En este caso no lleva ningún parámetro, el id de la canción va directamente en el url
response_ = requests.get(url_, params = {}, headers = headers_)
json_data_ = response_.json()

len_json = np.size(json_data_['items'])

for i in range(len_json):
  print(i,json_data_['items'][i]['name'])

0 Coconut Radio
1 Rock Argentino años 60
2 ROCK NACIONAL ARGENTINO || 2020
3  ROCK ARGENTINO 🔥 : Rock Nacional Argentino
4 Rock argentino 2000-2020
5 Rock argentino 2000
6 Rock Nacional Argentino 2000s
7 Rock Argentino 60’70’80’
8 ROCK ARGENTINO 70's
9 Rock Argentino 80's
10 Rock argentino 90's
11 Rock Nacional 80 y 90's
12 Mix rock nacional 80'90'
13 Rock nacional 60 y 70
14 Rock nacional '60 y '70 (y algo de los '80)
15 Rock nacional 60 70 80
16 Rock Nacional de los 70
17 Rock nacional 2010-2019
18 Rock Nacional 2000 - 2010
19 Rock Nacional 90's ARG
20 Rock Nacional 80's ARG
21 Rock Nacional 70/80’
22  ROCK NACIONAL ARGENTINO 
23 Best Classical Music
24 Guardians of the Galaxy Vol. 1&2 
25 Antidomingo
26 Top 50 - Argentina
27 Rock nac
28 Disparo al corazón
29 Shazam
30 Mis pistas en Shazam
31 Shazam (<2018)
32 All Out 70s
33 Abel Pintos — Unico (En Vivo)
34 MILO
35 Top 50 - Global
36 stefy
37 Classical World: Italy
38 Def Jam Classics: The Playlist
39 Dance Hits
40 Éxitos Argentina
4

In [ ]:
list1 = list(np.arange(26)) #Elijo listas random menos la del top50
list2 = [27,28,43]
list1.extend(list2)
print(list1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 43]


In [ ]:
#Elijamos algunas playlist que no sean el TOP 50
playlists = list1 #nro de playlists (Las ultimas playlists que descargue para el entrenamiento)
#playlists = [0,1,2,3,4] #Rock nacional
analyzer = spotydatos(CLIENT_ID,CLIENT_SECRET)
appended_df_lyrics = []
appended_df_features = []

for nro in playlists:
  playlist_id = json_data_['items'][nro]['id']
   
  try:
    
    #las tracks ID, nombres y artistas de las canciones de la playlist
    songs = analyzer.get_songs_on_playlist(playlist_id,only_id=False)
    songs = songs[0:-2] #Borramos la ultima cancion de cada playlist
    #hay muchas playlist con 100 canciones Y 100 CANCIONES ES EL LIMITE, nos quedamos con 99 y listo!
    songs_names = []
    songs_artists = []
    songs_id = []
    songs_total_tracks = []
    songs_track_number = []
    songs_disc_number = []
    songs_popularity = []
    songs_date = []
    songs_names_raw = [] #esto me va a servir para mas tarde
    songs_artists_raw = [] #esto me va a servir para mas tarde

    for i in range(np.size(songs)):
      songs_names.append(songs[i]['Track name'])
      songs_names_raw.append(songs[i]['Track name'])
      songs_artists.append(songs[i]['Artist'])
      songs_artists_raw.append(songs[i]['Artist'])
      songs_id.append(songs[i]['id'])
      songs_total_tracks.append(songs[i]['Total tracks'])
      songs_track_number.append(songs[i]['Track number'])
      songs_disc_number.append(songs[i]['Disc number'])
      songs_popularity.append(songs[i]['Popularity'])
      songs_date.append(songs[i]['Date'])
    #print(songs_id)

    #las features de las canciones
    features = []
    for j in range(np.size(songs_id)): #de nuevo, len(list) no funca! 
      features.append(analyzer.get_song_features(songs_id[j]))

    #Dataframe de features
    df_features_NO_top = pd.DataFrame(features, index=songs_id)
    df_features_NO_top.drop(df_features_NO_top['id'])
    df_features_NO_top['Disc number'] = songs_disc_number #agregamos features
    df_features_NO_top['Total tracks'] = songs_total_tracks
    df_features_NO_top['Track number'] = songs_track_number
    df_features_NO_top['Date'] = songs_date
    df_features_NO_top['Popularity'] = songs_popularity
    df_features_NO_top['TOP'] = 0 #No esta en el top
    df_features_NO_top['Artist'] = songs_artists_raw
    df_features_NO_top['Song'] = songs_names_raw
    appended_df_features.append(df_features_NO_top)
    
    del features
    del songs
    del df_features_NO_top

    
    #las letras
    url_base = 'https://www.musica.com/letras.asp?t2='
    urls = []
    songs_names_clean = []

    for song in range(np.size(songs_names)):
      songs_names[song] = songs_names[song].replace(' ','+').replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('Á','a').replace('É','e').replace('Í','i').replace('Ó','o').replace('Ú','u').lower()
      songs_artists[song] = songs_artists[song].replace(' ','+').replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('Á','a').replace('É','e').replace('Í','i').replace('Ó','o').replace('Ú','u').lower()
      split_1 = songs_names[song].split(':',1) #sacamos los (feat), remix, :, ...
      split_2 = split_1[0].split('+(',1)
      split_3 = split_2[0].split('(',1)
      split_4 = split_3[0].split('remix',1)
      split_5 = split_4[0].split('+-',1)
      songs_names_clean.append(split_5[0]) 
      #print(song, split_4[0])
      
      urls.append([url_base + songs_names_clean[song] +'+'+ songs_artists[song]])

    # Hacemos un request iterativo a la página de musica.com y sacamos como output los urls a las letras
    url_lyrics = []

    for url in np.arange(np.size(urls)):
      try:
        search = requests.get(urls[url][0])
        search_parseada = bs(search.content, 'html.parser') # Parseamos el contenido del request como un html
        # Soup
        soup = bs(response.content)
        tag_song = search_parseada.find(name = 'table', attrs = {'class' : 'rst'}) #tag madre
        tag_song = tag_song.find(name = 'a') # tag en donde esta el link a la letra
        url_lyrics.append(tag_song['href']) # el link en cuestion
        #print(url)
      except:
        url_lyrics.append(np.NaN) #le asigno un NaN
        #print(url, 'Lyrics not found')

    #iteramos sobre toda las urls de las letras
    lyrics = []
    for url in np.arange(np.size(url_lyrics)):
      try:
        search_song = requests.get(url_lyrics[url]) #pedimos el request
        search_song_parseada = bs(search_song.content, 'html.parser')

        body = search_song_parseada.find(name = 'div', attrs = {'id' : 'letra'}) #aca esta la letra, entre otras cosas

        letra_raw = body.find_all('p') #buscamos los paragraphs
        letra_raw_string = str(letra_raw) #to string
        letra_raw_string = str(letra_raw).replace('<br/>',' ').replace('</p>','').replace('<p>','') #mas bonita
        lyrics.append(letra_raw_string)
        #print(url)
      except:
        lyrics.append(np.NaN)
        #print(url,'Lyrics not found')

    #Dataframe lyrics
    df_lyrics_NO_top = pd.DataFrame(list(zip(songs_names_raw,songs_artists_raw,lyrics)),index=songs_id)
    df_lyrics_NO_top['TOP'] = 0 #No esta en el top
    appended_df_lyrics.append(df_lyrics_NO_top)


    print(nro, 'Playlist: {}'.format(json_data_['items'][nro]['name']),'(worked!)')
  except:
    print(nro, 'Playlist: {}'.format(json_data_['items'][nro]['name']),'(not working)')
    pass



0 Playlist: Coconut Radio (worked!)
1 Playlist: Rock Argentino años 60 (worked!)
2 Playlist: ROCK NACIONAL ARGENTINO || 2020 (worked!)
3 Playlist:  ROCK ARGENTINO 🔥 : Rock Nacional Argentino (worked!)
4 Playlist: Rock argentino 2000-2020 (worked!)
5 Playlist: Rock argentino 2000 (worked!)
6 Playlist: Rock Nacional Argentino 2000s (worked!)
7 Playlist: Rock Argentino 60’70’80’ (worked!)
8 Playlist: ROCK ARGENTINO 70's (worked!)
9 Playlist: Rock Argentino 80's (worked!)
10 Playlist: Rock argentino 90's (worked!)
11 Playlist: Rock Nacional 80 y 90's (worked!)
12 Playlist: Mix rock nacional 80'90' (worked!)
13 Playlist: Rock nacional 60 y 70 (worked!)
14 Playlist: Rock nacional '60 y '70 (y algo de los '80) (worked!)
15 Playlist: Rock nacional 60 70 80 (worked!)
16 Playlist: Rock Nacional de los 70 (worked!)
17 Playlist: Rock nacional 2010-2019 (worked!)
18 Playlist: Rock Nacional 2000 - 2010 (worked!)
19 Playlist: Rock Nacional 90's ARG (worked!)
20 Playlist: Rock Nacional 80's ARG (worke

In [ ]:
track_id = songs_id[0]
headers = {'Authorization': 'Bearer {}'.format(access_token), 'Accept': 'application/json', 'Content-Type': 'application/json'}
url = 'https://api.spotify.com/v1/audio-features/{}'.format(track_id)
response = requests.get(url, params = {}, headers = headers)
json_data = response.json()
json_data.keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [ ]:
#dataframe final de las features de las canciones que no estan en el TOP
df_no_top_features = pd.concat(appended_df_features)

#dataframe final de las letras de las canciones que no estan en el TOP
df_no_top_lyrics = pd.concat(appended_df_lyrics)

#mergeamos todo

df_FINAL_features = df_no_top_features.append(df_features_top)
df_FINAL_lyrics = df_no_top_lyrics.append(df_lyrics_top)

In [ ]:
df_FINAL_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,duration_ms,time_signature,Disc number,Total tracks,Track number,Date,Popularity,TOP,Artist,Song
3c8JemHol1XwFFruXQxjCO,0.496,0.368,4.0,-15.194,1.0,0.0338,0.460,0.000075,0.3110,0.628,128.331,audio_features,3c8JemHol1XwFFruXQxjCO,152227.0,4.0,1,12,1,1985-01-01,63,0,Various Artists,Everybody's Talkin'
6vxHp3CDNo0afgKGp2yi1E,0.626,0.816,9.0,-10.593,1.0,0.1310,0.237,0.000002,0.1040,0.798,105.307,audio_features,6vxHp3CDNo0afgKGp2yi1E,164813.0,4.0,1,14,6,1972-01-14,70,0,Paul Simon,Me and Julio Down by the Schoolyard
7CQXyFX44CXmnuq8Bi9Dyc,0.542,0.492,2.0,-11.927,1.0,0.0256,0.467,0.001050,0.0863,0.580,103.064,audio_features,7CQXyFX44CXmnuq8Bi9Dyc,279280.0,4.0,1,11,4,1967-03-10,55,0,The Youngbloods,Get Together
1CAO7hiNOxJRPW4nFv2aRO,0.673,0.673,1.0,-13.941,1.0,0.0493,0.373,0.026500,0.2060,0.825,119.994,audio_features,1CAO7hiNOxJRPW4nFv2aRO,198933.0,4.0,1,9,7,1970-01-01,64,0,Three Dog Night,Mama Told Me (Not To Come) - Single Version
3k63RLvRgkgPGx0keOH3P6,0.460,0.389,0.0,-11.603,1.0,0.0310,0.349,0.000000,0.0533,0.606,123.213,audio_features,3k63RLvRgkgPGx0keOH3P6,159973.0,4.0,1,12,7,1967-06-05,57,0,Stone Poneys,Different Drum


# Google Drive Mount

In [ ]:
#todo a .txt nomas! 

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df_FINAL_features.to_csv('df_FINAL_features_random.csv')
!cp df_FINAL_features_random.csv "drive/My Drive/LaboDatos/"

df_FINAL_lyrics.to_csv('df_FINAL_lyrics_random.csv')
!cp df_FINAL_lyrics_random.csv "drive/My Drive/LaboDatos/"

df_no_top_lyrics.to_csv('df_no_top_lyrics_random.csv')
!cp df_no_top_lyrics_random.csv "drive/My Drive/LaboDatos/"

df_lyrics_top.to_csv('df_lyrics_top_random.csv')
!cp df_lyrics_top_random.csv "drive/My Drive/LaboDatos/"
